Importing Libraries

In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from copy import deepcopy
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
print("Using device:", device)
if device.type == "cuda":
    print("CUDA device name:", torch.cuda.get_device_name(0))

True
Using device: cuda
CUDA device name: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [2]:
def to_device(tensor):
    return tensor.to(device)

Class for connect4 environment

In [ ]:
class Connect4Env:
    def __init__(self):
        self.rows = 6
        self.cols = 7
        self.reset()

    def reset(self):
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.current_player = 1
        self.last_move = None
        return self._get_obs()

    def _get_obs(self):
        p1_board = (self.board == 1).astype(np.float32)
        p2_board = (self.board == -1).astype(np.float32)
        if self.current_player == 1:
            return np.stack([p1_board, p2_board], axis=0)
        else:
            return np.stack([p2_board, p1_board], axis=0)

    def valid_actions(self):
        return [c for c in range(self.cols) if self.board[0, c] == 0]

    def step(self, action):
        if action not in self.valid_actions():
            return self._get_obs(), -10, True, {}

        for r in range(self.rows - 1, -1, -1):
            if self.board[r, action] == 0:
                self.board[r, action] = self.current_player
                self.last_move = (r, action)
                break

        if self._check_winner(self.current_player, self.last_move):
            return self._get_obs(), 1, True, {}

        if len(self.valid_actions()) == 0:
            return self._get_obs(), 0, True, {}

        self.current_player *= -1
        return self._get_obs(), -0.01, False, {}

    def _check_winner(self, player, last_move):
        if last_move is None:
            return False
        r, c = last_move
        directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
        for dr, dc in directions:
            count = 1
            for d in [-1, 1]:
                nr, nc = r + d * dr, c + d * dc
                while 0 <= nr < self.rows and 0 <= nc < self.cols and self.board[nr, nc] == player:
                    count += 1
                    if count >= 4:
                        return True
                    nr += d * dr
                    nc += d * dc
        return False



Model

In [4]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 6 * 7, 128)
        self.out = nn.Linear(128, 7)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.out(x)

Replay Buffer

In [5]:
class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)

    def push(self, transition):
        self.buffer.append(transition)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

Selecting Action

In [6]:
# --- Training Functions ---
def select_action(model, state, epsilon, valid_actions):
    if random.random() < epsilon:
        return random.choice(valid_actions)
    else:
        with torch.no_grad():
            q_values = model(torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device))
            q_values = q_values.squeeze()
            q_values[[i for i in range(7) if i not in valid_actions]] = -float('inf')
            return torch.argmax(q_values).item()

Training Loop

In [ ]:
def train_self_play(
    episodes=50000, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.995,
    gamma=0.99, batch_size=64, update_target_every=10
):
    env = Connect4Env()
    model1 = DQN().to(device)
    model2 = DQN().to(device)
    target_model1 = DQN().to(device)
    target_model2 = DQN().to(device)
    target_model1.load_state_dict(model1.state_dict())
    target_model2.load_state_dict(model2.state_dict())
    optimizer1 = optim.Adam(model1.parameters(), lr=1e-3)
    optimizer2 = optim.Adam(model2.parameters(), lr=1e-3)
    buffer1 = ReplayBuffer()
    buffer2 = ReplayBuffer()

    for episode in range(episodes):
        state = env.reset()
        done = False
        reward_p1 = 0
        reward_p2 = 0
        last_moves = {1: None, -1: None}  # track last move per player

        while not done:
            current_player = env.current_player
            model = model1 if current_player == 1 else model2
            buffer = buffer1 if current_player == 1 else buffer2
            optimizer = optimizer1 if current_player == 1 else optimizer2
            target_model = target_model1 if current_player == 1 else target_model2

            valid_actions = env.valid_actions()
            action = select_action(model, state, epsilon, valid_actions)

            next_state, reward, done, _ = env.step(action)

            # Store last move for possible later penalty
            last_moves[current_player] = (np.array(state, copy=True), action, np.array(next_state, copy=True))

            # Store current player's experience
            buffer.push((np.array(state, copy=True), action, reward, np.array(next_state, copy=True), done))
            state = next_state

            # Track cumulative reward
            if current_player == 1:
                reward_p1 += reward
                if done and reward == 1 and last_moves[-1] is not None:
                    reward_p2 += -1.0  # opponent lost
            else:
                reward_p2 += reward
                if done and reward == 1 and last_moves[1] is not None:
                    reward_p1 += -1.0  # opponent lost

            # If game is over and current player won, penalize opponent
            if done and reward == 1:
                losing_player = -current_player
                if last_moves[losing_player] is not None:
                    s, a, ns = last_moves[losing_player]
                    opp_buffer = buffer1 if losing_player == 1 else buffer2
                    opp_buffer.push((s, a, -1.0, ns, True))

            # Train on batch if buffer is ready
            if len(buffer) >= batch_size:
                transitions = buffer.sample(batch_size)
                states, actions, rewards, next_states, dones = zip(*transitions)

                states = torch.tensor(np.stack(states), dtype=torch.float32).to(device)
                actions = torch.tensor(actions).unsqueeze(1).to(device)
                rewards = torch.tensor(rewards).unsqueeze(1).to(device)
                next_states = torch.tensor(np.stack(next_states), dtype=torch.float32).to(device)
                dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1).to(device)

                q_values = model(states).gather(1, actions)
                with torch.no_grad():
                    max_next_q = target_model(next_states).max(1)[0].unsqueeze(1)
                    target_q = rewards + gamma * max_next_q * (1 - dones)

                loss = F.mse_loss(q_values, target_q)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # Epsilon decay
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

        # Update target models periodically
        if episode % update_target_every == 0:
            target_model1.load_state_dict(model1.state_dict())
            target_model2.load_state_dict(model2.state_dict())

        # Logging
        if episode % 100 == 0:
            print(f"Episode {episode}, P1 reward: {reward_p1:.2f}, P2 reward: {reward_p2:.2f}, Epsilon: {epsilon:.3f}")

        # Periodic saving
        if episode % 10000 == 0 and episode > 0:
            torch.save(model1.state_dict(), f"connect4_model1_ep{episode}.pth")
            torch.save(model2.state_dict(), f"connect4_model2_ep{episode}.pth")

    # Save final models
    torch.save(model1.state_dict(), "connect4_model1_final.pth")
    torch.save(model2.state_dict(), "connect4_model2_final.pth")

    return model1, model2


Train It

In [20]:
trained_model1, trained_model2 = train_self_play(episodes=100000,epsilon_decay=0.9995)

Penalized Player -1 for losing
Episode 0, P1 reward: 0.84, P2 reward: -1.16, Epsilon: 1.000
Penalized Player 1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player 1 for losing
Penalized Player 1 for losing
Penalized Player 1 for losing
Penalized Player 1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player 1 for losing
Penalized Player 1 for losing
Penalized Player 1 for losing
Penalized Player -1 for losing
Penalized Player 1 for losing
Penalized Player 1 for losing
Penalized Player -1 for losing
Penalized Player 1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player 1 for losing
Penalized Player 1 for losing
Penalized Player -1 for losing
Penalized Player -1 for losing
Penalized Player 1 for losing
Penalized Player -1 for losing
Penalized Player -1 for

KeyboardInterrupt: 

In [8]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [9]:
save_model(trained_model)

NameError: name 'trained_model' is not defined

Load Weights

In [10]:
def load_trained_model(path="connect4_model_weights.pth"):
    model = DQN().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()  # Set to evaluation mode
    print(f"Loaded model from {path}")
    return model

In [11]:
trained_model = load_trained_model(path="connect4_dqn_ep10000.pth")

Loaded model from connect4_dqn_ep10000.pth


Playing

In [15]:
def play_against_model(starting_player=None):
    model_choice = input("Play against model 1 (X) or model 2 (O)? Enter 1 or 2: ").strip()
    model_path = "connect4_model1_ep20000.pth" if model_choice == "1" else "connect4_model2_ep20000.pth"
    model = DQN().to(device)
    env = Connect4Env()

    model.load_state_dict(torch.load(model_path))
    model.eval()

    state = env.reset()
    done = False

    if starting_player is None:
        choice = input("Do you want to go first? (y/n): ").lower()
        if choice == 'y':
            env.current_player = -1
        else:
            env.current_player = 1
    else:
        env.current_player = -1 if starting_player == -1 else 1
    print("You are Player -1 (O). Model is Player 1 (X). Columns: 0 to 6")
    print(f"Playing against model from: {model_path}")
    print(env.board)

    while not done:
        if env.current_player == -1:
            try:
                user_action = int(input("Your move (0-6): "))
            except ValueError:
                print("Invalid input. Try a number from 0 to 6.")
                continue
            if user_action not in env.valid_actions():
                print("Invalid move. Try again.")
                continue
            state, reward, done, _ = env.step(user_action)
        else:
            valid_actions = env.valid_actions()
            action = select_action(model, state, epsilon=0.0, valid_actions=valid_actions)
            state, reward, done, _ = env.step(action)
            print("Model played column:", action)

        # Always print board after any move
        symbol_map = {1: 'X', -1: 'O', 0: '.'}
        print("\nCurrent board:")
        for row in env.board:
            print(' '.join(symbol_map[cell] for cell in row))
        print()

        if done:
            print("Game Over. Reward:", reward)
            break



In [16]:
play_against_model()

C:\Users\ajmat\AppData\Local\Temp\ipykernel_77264\3961894182.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


You are Player -1 (O). Model is Player 1 (X). Columns: 0 to 6
Playing against model from: connect4_model2_ep20000.pth
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Model played column: 4

Current board:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .


Current board:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O X . .

Model played column: 1

Current board:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . O X . .


Current board:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. X . O X . .

Model played column: 4

Current board:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O X . .
. X . O X . .


Current board:
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . O X . .
. X . O X . .

Model played column: 1

Current board:
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. X . O X